<style>
    h1 {
        color: #178773; 
        font-size: 50px !important;
        text-align: center;
    }
    h2 {
    	background-color: #178773;
    	color: white;
    	font-size: 25px !important;
    	font-weight: 300 !important;
    	padding: 5px;
        text-align: center;
    }
    h3 {
        color: #178773; 
        font-size: 20px !important;
        font-weight: 500 !important;
        text-align: center;
    }   
    .jp-RenderedImage {
        display: flex;
        justify-content: center;
    }
    
    .navbar {/*display: none*/}
    
    h4 {
    	color: #178773;
    	font-size: 34px !important;
    	font-weight: 900 !important;
    	margin-bottom: 25px;
    }
    .jp-Notebook {
        max-width: 1220px;
        justify-content: center;
        display: flex;
        margin: auto;
      }
    
    .ksln-grid {
      justify-content: left;
      display: grid;
      margin-bottom: 50px;	
      gap: 25px;
      grid-template-columns: 100%;
      max-width: 95%;  
    }
    .ksln-grid-2c {
      justify-content: left;
      display: grid;
      margin: 50px 0px;	
      gap: 25px;
      grid-template-columns: 50% 50%;
      max-width: 95%;  
    }

    .ksln-cards {
        border: 0.05rem solid var(--md-default-fg-color--lightest);
        border-radius: 0.1rem;
        padding-bottom: 10px;
        width: 100%;
        box-shadow: 0px 0px 3px rgb(0 0 0 / 15%);
        font-weight: 300;
        background-color: var(--md-default-bg-color);
    }

    .ksln-cards:hover {
        box-shadow: 0px 0px 10px rgb(0 0 0 / 30%); /* Ombre agrandie au survol */
    }
    
    @media only screen and (min-width: 769px) and (max-width: 1399px) {
    .ksln-grid {
      grid-template-columns: 50% 50%;
    }
    }
    @media only screen and (min-width: 1400px) {
    .ksln-grid {
      grid-template-columns: 33.33337% 33.33337% 33.33337%;
    }
    }
</style>

In [1]:
import mercury as mr
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import textwrap
from textwrap import wrap
from IPython.display import display, HTML, Markdown, Javascript
from types import SimpleNamespace
import os


app = mr.App(title="Diag360 - Outil de visualisation - V5", description="Tableaux de bord des besoins et enjeux de résilience du territoire.")

mercury.App

In [2]:
# ==== FONCTIONS & PARAMETRAGE (hors loop-widgets)

choice_enjeux = 'Diag360°'

choice_objectifs = ['Transformation',
                   'Subsistance',
                   'Soutenabilité',
                   'Gestion de crise']
i_gen = 1

# Fonction pour tronquer et ajouter des retours à la ligne aux textes trop longs sans couper les mots
def format_label(text, max_len=120, line_len=40):
    if len(text) > max_len:
        text = text[:max_len] + '...'
    wrapped_text = textwrap.fill(text, width=line_len, break_long_words=False)
    return wrapped_text.replace('\n', '<br>')

def add_to_radar(df, groupe, s_groupe):

    # Regrouper les données par type de besoins et besoins
    df_grouped = df.groupby([groupe,s_groupe])["valeur_indice"].mean().reset_index()
    # Ajouter une colonne pour identifier les groupes "A", "B", "C", etc.
    df_grouped["Indice"] = (df_grouped[groupe] != df_grouped[groupe].shift()).cumsum().map(lambda x: chr(64 + x))
    # Cumulative length
    values = df_grouped["valeur_indice"].values

    # Initialize layout in polar coordinates
    # fig, ax = plt.subplots(figsize=(4.5, 4.5), subplot_kw={"projection": "polar"})
    fig, ax = plt.subplots(figsize=(5, 5), subplot_kw=dict(polar=True))

    # Parametres d'espace
    labels = df_grouped[s_groupe].values
    values = df_grouped["valeur_indice"].values
    angles = np.linspace(0, 2 * np.pi, len(df_grouped), endpoint=False)
    num_vars = len(labels)
    
    # Normalisation des valeurs entre 0 et 1
    norm = Normalize(vmin=0, vmax=1)
    # Choisir une colormap
    cmap = plt.get_cmap('RdYlGn')  # 'RdYlGn' va du rouge (faible valeur) au vert (haute valeur)
    # Convertir les valeurs en couleurs
    colors = cmap(norm(values))
    
    # ================ PLOT & Some layout stuff ================
    # Change the color of the outermost gridline (the spine).
    ax.spines['polar'].set_color('white')
    # Fix axis to go in the right order and start at 12 o'clock.
    ax.set_theta_offset(np.pi / 2)
    #ax.set_theta_direction(-1)
    ax.set_thetagrids(np.degrees(angles), labels)
    
    
    # ======= AUTRES ELEMENTS DE DESIGN
    # Change the color of the outermost gridline (the spine).
    ax.spines['polar'].set_color('white')
    # Change the background color inside the circle itself.
    ax.set_facecolor('#FAFAFA')
    # ax.fill(angles, values, color='#0b927b', alpha=0.3, zorder=11)
    # fig.patch.set_facecolor("white")
    
    
    # ======== LABELS ========
    # Go through labels and adjust alignment based on where it is in the circle.
    # Go through labels and adjust alignment based on where it is in the circle.
    for label, angle in zip(ax.get_xticklabels(), angles):
      if angle in (0, np.pi):
        label.set_horizontalalignment('center')
      elif 0 < angle < np.pi:
        label.set_horizontalalignment('right')
      else:
        label.set_horizontalalignment('left')

    
    ax.set_axisbelow(False)
    
    
    # ======= AXE X =========
    labels = ["\n".join(wrap(r[:38] + ('...' if len(r) > 38 else ''), 22, break_long_words=False)) for r in labels]
    ax.set_xticks(angles)
    ax.set_xticklabels(labels)
    ax.tick_params(axis='x',labelfontfamily='Dosis' , labelsize=16-0.25*num_vars, labelcolor='#222', grid_color='#555', grid_alpha=0.1, pad=3)
    # Ploting the bars to represent the cumulative track
    ax.bar(angles, values, color=colors, alpha=1, width=2*np.pi/num_vars, zorder=1, linewidth=3, edgecolor='white')
    
                  
    # ======= AXE Y =========
    # Make the y-axis (0-100) labels smaller.
    ax.tick_params(axis='y', labelcolor='white', labelsize=0, grid_color='#FFF', grid_alpha=0, width=0)
    ax.set_ylim(0, 1.05)


# Function to truncate description
def truncate_text(text, max_length=400):
    return text if len(text) <= max_length else text[:max_length] + ' [...]'


In [ ]:
# ==== ACCUEIL & TELECHARGEMENT

# Ajouter un widget de téléchargement de fichier
my_file = mr.File(label="Téléchargement de fichier")
url = my_file.filepath

texte = """<div style="margin-top: 100px; background-color: #FAFAFA; max-width: 100%"><div class="ksln-cards" style="margin: 2px; padding: 10px;"><h1>Vous débutez avec Diag360 ?</h1>\n\n1. Télécharger ce fichier et suivez les instructions, nous vous retrouvons ici juste après 🤗 : [Télécharger le tableur de données](https://github.com/Konsilion/diag360/raw/master/mkdocs/media/Diag360_Indicateurs.xlsx)
\n2. Une fois le fichier remplis, cliquer sur le **bouton 'Browse' à gauche**, et séléctionnez votre tableur de travail.
\n\nPour plus d'information, rendez-vous sur le site suivant : [Documentation Diag360](https://konsilion.github.io/diag360/)</div></div>"""

# Charger les données du fichier Excel
# url = '/home/nicolasbremond/Documents/Local_Konnect/Diag360/Diag360_Indicateurs_V4.xlsx'
#url = 'https://raw.githubusercontent.com/Konsilion/diag360/master/mkdocs/media/Diag360_Indicateurs_V4.xlsx'

if url is None:
    display(Markdown(texte))
    mr.Stop()

# Charger les données du fichier Excel depuis la feuille "Export"
df = pd.read_excel(url, sheet_name='Export')
df_besoins = pd.read_excel(url, sheet_name="Besoins_Infos")
df_indicateurs = pd.read_excel(url, sheet_name="Indicateurs_Infos")

In [ ]:
# ==== WIDGETS

# Choix type suivi : Sunburst ou Cible
mapping_plot = {
    'Panorama 360°': 'interactif',
    'Synthèse': 'synthese'
}

# Liste des clés du dictionnaire pour le menu déroulant
list_ticker = list(mapping_plot.keys())
choice_plot = 'Panorama 360°'
ticker_plot = mr.Select(value='Panorama 360°', choices=list_ticker, label="Type de représentation :")
choice_plot = mapping_plot[ticker_plot.value]

ticker_pilot = SimpleNamespace(value='Panorama de la Résilience territoriale')


path_value=['type_besoins', 'besoins', 'objectif', 'designation_indicateur']

if (choice_objectifs != []):
    path_value=['type_besoins', 'besoins', 'objectif', 'designation_indicateur']
else:
    # Filtrer la DataFrame selon choice_objectifs
    df = pd.read_excel(url, sheet_name='Export')
    path_value=['type_besoins', 'besoins', 'designation_indicateur']








if (choice_plot != "interactif"):
    mapping_pilot = {
        'Besoins du territoire': 'besoins',
        'Indicateurs de pilotage': 'designation_indicateur',
    }
    
    # Liste des clés du dictionnaire pour le menu déroulant
    list_ticker = list(mapping_pilot.keys())
    choice_pilot = 'Besoins du territoire'
    ticker_pilot = mr.Select(value='Besoins du territoire', choices=list_ticker, label="Afficher les :")
    choice_pilot = mapping_pilot[ticker_pilot.value]






#if (choice_plot != "interactif"):
mapping_enjeux = {
    'Vitaux': 'Vitaux',
    'Essentiels': 'Essentiels',
    'Induits': 'Induits',
    'Vitaux, Essentiels et Induits': 'Tous'
}
# Liste des clés du dictionnaire pour le menu déroulant
list_ticker = list(mapping_enjeux.keys())
choice_enjeux = 'Vitaux, Essentiels et Induits'
ticker = mr.Select(value='Vitaux, Essentiels et Induits', choices=list_ticker, label="Concernant les enjeux :")
choice_enjeux = mapping_enjeux[ticker.value]

# ==== FILTRE SUR LES VALEURS : type_besoin

if choice_enjeux != 'Tous':   
    # Filtrer la DataFrame selon choice_enjeux
    df = pd.read_excel(url, sheet_name='Export')
    df = df[df['type_besoins'] == choice_enjeux]
    df = df[df['objectif'].isin(choice_objectifs)]
else:
    # Filtrer la DataFrame selon choice_enjeux
    df = pd.read_excel(url, sheet_name='Export') 

if (choice_enjeux != 'Tous'):
    path_value=['besoins', 'objectif', 'designation_indicateur']


#if (choice_plot != "interactif"):
# Liste des clés du dictionnaire pour le menu déroulant
list_ticker = ['Transformation',
               'Subsistance',
               'Soutenabilité',
               'Gestion de crise']
choice_objectifs = ["Transformation"]
#ticker = mr.Select(value='Tous', choices=list_ticker, label="Selectionnez les objectifs territoriaux :")
ticker = mr.MultiSelect(label = "avec pour objectifs :", value = list_ticker, choices = list_ticker)
choice_objectifs = list(ticker.value)

# ==== FILTRE SUR LES VALEURS : objectif

if choice_objectifs != []:   
    # Filtrer la DataFrame selon choice_objectifs
    df = pd.read_excel(url, sheet_name='Export')
    if choice_enjeux != 'Tous':   
        # Filtrer la DataFrame selon choice_enjeux
        df = df[df['type_besoins'] == choice_enjeux]
    df = df[df['objectif'].isin(choice_objectifs)]
    path_value=['type_besoins', 'besoins', 'objectif', 'designation_indicateur']
else:
    # Filtrer la DataFrame selon choice_objectifs
    df = pd.read_excel(url, sheet_name='Export')
    path_value=['type_besoins', 'besoins', 'designation_indicateur']
    if choice_enjeux != 'Tous':   
        # Filtrer la DataFrame selon choice_enjeux
        df = df[df['type_besoins'] == choice_enjeux]  


In [34]:
# ==== FORMATAGE DES DONNEES

# Filtrer uniquement les lignes où 'valeur_indice' est une valeur numérique
df = df[pd.to_numeric(df['valeur_indice'], errors='coerce').notnull()]

# Convertir 'valeur_indice' en numérique
df['valeur_indice'] = pd.to_numeric(df['valeur_indice'])

# Vérifier les données pour les valeurs nulles ou zéro
df = df[df['valeur_indice'] > 0]

# Ajouter une colonne pour uniformiser les valeurs au niveau supérieur
df['uniform_value'] = 1

# Appliquer la fonction de formatage aux labels
# df['designation_indicateur'] = df['designation_indicateur'].apply(format_label)

In [ ]:
# ==== TITRE H2 & PRESENTATION DIAG360

# Texte en une seule ligne de markdown
texte = """# **Tour d'horizon de votre résilience territoriale**"""
display(Markdown(texte))

if choice_enjeux == "Tous":
    var_titre = "Vitaux, Essentiels et Induits"
else:
    var_titre = choice_enjeux
    
texte_h2 = "## " + ticker_pilot.value + " - " + var_titre
display(Markdown(texte_h2))

texte = "### sur objectif de : " + ", ".join(choice_objectifs) + "<hr>"
if choice_objectifs == []:
    texte = "### sur objectif : de Transformation, Subsistance, Soutenabilité, Gestion de crise"

display(Markdown(texte))

In [ ]:
# ==== PLOT : SUNBURST

def ksln_sunburst():
    # Créer le graphique Sunburst
    colorscale = [
    [0, '#CA0D00'], 
    [0.5, '#FCF68E'],
    [1, '#05892F']
    ]
    
    fig = px.sunburst(
        df,
        path=path_value,
        values='uniform_value',  # Utiliser des valeurs uniformes pour égaliser les catégories principales
        color='valeur_indice',
        color_continuous_scale=colorscale
    )
    
    # Mettre à jour la taille du graphique
    fig.update_layout(
        showlegend=False,
        margin=dict(t=0, l=100, r=0, b=0)
    )
    
    fig.show()

if choice_plot == "interactif":    
    ksln_sunburst()
else:
    add_to_radar(df,'type_besoins',choice_pilot)



         

In [ ]:

html_content = '<hr style="margin: 50px 0 !important;"><div class="ksln-grid">\n'

if (choice_plot != "interactif"):
    if choice_pilot == 'besoins':
        # Generate HTML content
        for index, row in df_besoins.iterrows():
            if row['besoins'] in df['besoins'].values:
                html_content += f"""
                <a style="color: black; text-decoration: none;" href="{row['lien']}" target="_blank">
                    <div class="ksln-cards" style="margin: 0px auto auto 3px;">
                        <p style="text-align: center; font-weight: bold; padding: 10px; background-color: #ebebeb;">{row['besoins']}</p>
                            <p style="text-align: center; margin: 10px;">{truncate_text(row['description'])}</p>
                    </div>
                </a>
                """
    else:
        # Generate HTML content
        for index, row in df_indicateurs.iterrows():
            if row['designation_indicateur'] in df['designation_indicateur'].values:
                html_content += f"""
                <a style="color: black; text-decoration: none;" href="{row['lien']}" target="_blank">
                    <div class="ksln-cards" style="margin: 0px auto auto 3px;">
                        <p style="text-align: center; font-weight: bold; padding: 10px; background-color: #ebebeb;">{row['designation_indicateur']}</p>
                            <p style="text-align: center; margin: 10px;">{truncate_text(row['description'])}</p>
                    </div>
                </a>
                """
else:
    # Generate HTML content
    for index, row in df_besoins.iterrows():
        if row['besoins'] in df['besoins'].values:
            html_content += f"""
            <a style="color: black; text-decoration: none;" href="{row['lien']}" target="_blank">
                <div class="ksln-cards" style="margin: 0px auto auto 3px;">
                    <p style="text-align: center; font-weight: bold; padding: 10px; background-color: #ebebeb;">{row['besoins']}</p>
                        <p style="text-align: center; margin: 10px;">{truncate_text(row['description'])}</p>
                </div>
            </a>
            """
    
    
html_content += '</div>'
    
# Display the HTML content
display(HTML(html_content))